In [5]:
import h2o
from h2o.estimators.word2vec import H2OWord2vecEstimator
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
from git import Repo
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser
from pprint import pprint
from datetime import datetime
import random
from feature_engine.imputation import MeanMedianImputer
from sklearn.cluster import KMeans

In [190]:
okhttp_commits_df = pd.read_csv('raw_commits_github_json/Okhttp/Okhttp_commits.csv')

In [191]:
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring
0,3ef564b0ddcb00655e540e52ad882a5881c1dc54,Jesse Wilson,jesse@swank.ca,2021-11-01 13:29:31,swankjesse,Don't document mockwebserver3.internal (#6895),1ed9863131c87420381d8a6be48b96157a9bb764,2021-10-29 07:39:56,77.0,0
1,1ed9863131c87420381d8a6be48b96157a9bb764,Jesse Wilson,jesse@swank.ca,2021-10-29 07:39:56,swankjesse,Upgrade to Okio 3.0.0 (#6896),bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738,2021-10-22 16:51:49,158.0,0
2,bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738,Jim Hurne,hurne_jim@yahoo.com,2021-10-22 16:51:49,jthurne,Fix EventSources.processResponse to not throw ...,1ff16232f86d2b302840a3bb2f9d386e94374753,2021-10-15 05:04:08,179.0,0
3,1ff16232f86d2b302840a3bb2f9d386e94374753,Clément Dessoude,clement.dessoude@gmail.com,2021-10-15 05:04:08,clementdessoude,fix(docs): update broken link on docs homepage...,a87f4b4585fe6e8e49c7760986fd5d2c591e0573,2021-10-01 01:13:35,339.0,0
4,a87f4b4585fe6e8e49c7760986fd5d2c591e0573,Jesse Wilson,jwilson@squareup.com,2021-10-01 01:13:35,swankjesse,Changelog for OkHttp 4.9.2,286187102848b30d9580794795746b28462a79d0,2021-09-24 06:13:58,162.0,0


In [192]:
len(okhttp_commits_df)

4593

In [193]:
okhttp_commits_df = okhttp_commits_df[okhttp_commits_df['time_taken'] > 5]
okhttp_commits_df = okhttp_commits_df[okhttp_commits_df['time_taken'] < 168]
len(okhttp_commits_df)

2187

In [194]:
okhttp_commits_df['message'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

In [195]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,16 hours 4 mins
H2O_cluster_timezone:,Asia/Singapore
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.7
H2O_cluster_version_age:,2 months and 7 days
H2O_cluster_name:,H2O_from_python_tanji_ebw7bu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.976 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [196]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [197]:
STOP_WORDS = stopwords.words('english')
STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can","lines","re","what",
               "there","all","we","one","the","a","an","of","or","in","for","by","on",
               "but","is","in","a","not","with","as","was","if","they","are","this","and","it","have",
               "from","at","my","be","by","not","that","to","from","com","org","like","likes","so"]
STOP_WORDS = ['']



In [198]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [199]:
def create_token_table(data_hdf, col_name, prefix, sent_sample_rate = 0.0, epochs = 25, vec_size=20):
    # Tokenize
    words = tokenize(data_hdf[col_name])
    w2v_model = H2OWord2vecEstimator(sent_sample_rate = sent_sample_rate, 
                                     epochs = epochs, 
                                     vec_size = vec_size)
    w2v_model.train(training_frame=words)
    
    # Create the training vectors
    vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")
    
    
    # Prefix naming the columns
    vecs.columns = ['%s%s' % (prefix, c) for c in vecs.columns]
    data2_hdf = data_hdf.cbind(vecs)
    return data2_hdf, w2v_model

    
#     data3_hdf = data2_hdf[mask,:].cbind(vecs[mask,:])

In [200]:
commit_message_df = okhttp_commits_df[['message',]]

In [201]:
commit_message_df.shape

(2187, 1)

In [202]:
commit_message_df.head()

,message
0,Don't document mockwebserver3.internal (#6895)
1,Upgrade to Okio 3.0.0 (#6896)
4,Changelog for OkHttp 4.9.2
5,JDK 17 CI Build (#6864)
6,Update dependencies including kotlin 1.5.31 (#...


In [203]:
commit_message_hdf = h2o.H2OFrame(commit_message_df, 
                         column_types = {'message': 'string'}
                        )
commit_message_hdf.types

Parse progress: |█████████████████████████████████████████████████████████| 100%


{'message': 'string'}

In [204]:
commit_message_hdf.head()

message
Don't document mockwebserver3.internal (#6895)
Upgrade to Okio 3.0.0 (#6896)
Changelog for OkHttp 4.9.2
JDK 17 CI Build (#6864)
Update dependencies including kotlin 1.5.31 (#6865)
Update dependencies (#6850)
Replace deprecated method used in README (#6863)
Specify languages in kdoc code fragments (#6844)
"Fixed leading spaces for code blocks in recipes.md Markdown coverts text with four leading spaces into a code block. Fenced code blocks in this file had four leading spaces in addition to the being wrapped with ```, because of this the code blocks weren't being rendered properly in the markdown preview. Removing the two out of the four leading spaces fixed the issue."
Declare types & Cleanup (#6842) * Declare types * Code cleanup * Declare types


In [205]:
# Train owner names model
commit_message_vecs_hdf, commit_message_w2v_model = create_token_table(commit_message_hdf,
                                                                  'message',
                                                                   prefix='message_',
                                                                   vec_size=30)

'''
words = tokenize(commit_message_hdf['message'])
w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 20, 
                                 vec_size = 30)
w2v_model.train(training_frame=words)

# Create the training vectors
vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")


# Prefix naming the columns
vecs.columns = ['%s%s' % ('message_', c) for c in vecs.columns]
data2_hdf = commit_message_hdf.cbind(vecs)
print(data2_hdf)

wv_path = 'models/'
model_path = h2o.save_model(model = w2v_model, path= wv_path ,force=True)

#Load Model
w2v_model2 = h2o.load_model(model_path)
'''

word2vec Model Build progress: |██████████████████████████████████████████| 100%


'\nwords = tokenize(commit_message_hdf[\'message\'])\nw2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 20, \n                                 vec_size = 30)\nw2v_model.train(training_frame=words)\n\n# Create the training vectors\nvecs = w2v_model.transform(words, aggregate_method = "AVERAGE")\n\n\n# Prefix naming the columns\nvecs.columns = [\'%s%s\' % (\'message_\', c) for c in vecs.columns]\ndata2_hdf = commit_message_hdf.cbind(vecs)\nprint(data2_hdf)\n\nwv_path = \'models/\'\nmodel_path = h2o.save_model(model = w2v_model, path= wv_path ,force=True)\n\n#Load Model\nw2v_model2 = h2o.load_model(model_path)\n'

In [206]:
commit_message_w2v_model.find_synonyms('refactor')

OrderedDict([('huffman', 0.6406428813934326),
             ('realconnection', 0.6397698521614075),
             ('accept', 0.5950826406478882),
             ('realwebsocket', 0.5749762654304504),
             ('internal', 0.5693228244781494),
             ('upgrades', 0.5658422708511353),
             ('each', 0.5656113028526306),
             ('split', 0.5479177832603455),
             ('hide', 0.5433359742164612),
             ('javadocs', 0.5270000100135803),
             ('util', 0.5129557847976685),
             ('encoder', 0.5094821453094482),
             ('moves', 0.507489800453186),
             ('final', 0.5069358944892883),
             ('handshakecertificates', 0.5021960735321045),
             ('rename', 0.5002060532569885),
             ('convert', 0.4997645914554596),
             ('formencodingbuilder', 0.4923614263534546),
             ('classes', 0.4921002984046936),
             ('provided', 0.4886670708656311)])

In [207]:
rand_list = sorted( random.sample(range(commit_message_vecs_hdf.nrow), 4 ) )
commit_message_vecs_hdf[rand_list, :]

message,message_C1,message_C2,message_C3,message_C4,message_C5,message_C6,message_C7,message_C8,message_C9,message_C10,message_C11,message_C12,message_C13,message_C14,message_C15,message_C16,message_C17,message_C18,message_C19,message_C20,message_C21,message_C22,message_C23,message_C24,message_C25,message_C26,message_C27,message_C28,message_C29,message_C30
"Recover from an already-allocated connection that restricts new streams. (#3525) This is an edge case that can occur with HTTP/2. Since multiple requests use the same connection, it's possible for one request to restrict the connection from creating new streams during a follow-up request. https://github.com/square/okhttp/issues/3521",-0.0420357,0.0603334,0.0528032,0.129365,0.00160956,-0.295969,0.022628,0.264748,0.143842,-0.146443,-0.0502476,0.339285,0.137619,0.0498595,0.239793,0.0268989,-0.0201621,0.194123,-0.0531066,0.226358,-0.00528618,0.4896,-0.158539,-0.260634,-0.0203621,-0.123968,-0.197967,0.133694,-0.26649,-0.196222
Merge pull request #1875 from square/jw/body-websocket Represent WebSocket payload using RequestBody and ResponseBody.,-0.0823719,0.0852203,0.136524,-0.142039,0.243667,0.326504,0.306469,0.315736,0.372189,-0.388927,0.186375,0.269805,0.0301424,0.17589,0.353713,-0.28098,-0.16674,0.0658441,-0.0748462,0.429955,-0.141981,0.220424,-0.369717,-0.42829,-0.282609,-0.302807,-0.329459,-0.445196,-0.0893101,-0.0948916
Merge pull request #5481 from yschimke/platform_cleanup Minor Platform cleanup,-0.0566776,-0.184738,-5.13047e-05,-0.4157,0.293741,0.199166,0.264789,0.335806,0.432358,-0.744242,0.501359,0.401221,-0.284253,-0.0772747,0.208312,-0.417578,-0.14643,-0.297624,-0.512282,0.577027,0.0571759,0.143824,-0.345268,-0.473932,0.121271,0.0440744,-0.326733,-0.362262,0.0932669,-0.250326
Add another connection error message,0.228062,-0.345817,0.278343,0.264678,0.171791,-0.13559,0.0727545,0.0918429,0.207956,-0.177681,-0.0544239,0.634653,0.443901,0.0830168,-0.033617,-0.054879,-0.0948981,0.160199,0.00155633,0.12903,-0.0912828,0.370287,-0.0348202,-0.183047,-0.0689963,-0.373422,-0.119336,-0.339311,0.0513911,-0.345832


In [208]:
kmean = KMeans(n_clusters=10)

In [209]:
commit_message_vecs_hdf = commit_message_vecs_hdf.as_data_frame()

In [210]:
commit_message_vecs_hdf = commit_message_vecs_hdf.fillna(0)

In [211]:
kmean.fit(commit_message_vecs_hdf.iloc[:,1:])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\p

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\p

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\p

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\p

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\p

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\p

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\p

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\p

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float32
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\p

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [212]:
kmean.labels_

array([3, 4, 9, ..., 0, 3, 1])

In [213]:
commit_message_vecs_hdf['kmean_label'] = kmean.labels_

In [214]:
commit_message_vecs_hdf.head()

,message,message_C1,message_C2,message_C3,message_C4,message_C5,message_C6,message_C7,message_C8,message_C9,...,message_C22,message_C23,message_C24,message_C25,message_C26,message_C27,message_C28,message_C29,message_C30,kmean_label
0,Don't document mockwebserver3.internal (#6895),0.204399,0.287637,0.616095,-0.123909,0.287664,-0.127145,-0.100983,0.519332,0.306687,...,0.134050,0.110423,0.074149,0.166045,-0.336496,-0.230301,-0.017538,-0.051952,-0.372986,3
1,Upgrade to Okio 3.0.0 (#6896),0.176079,-0.235433,0.011171,0.796167,0.138725,0.242999,0.034900,0.510765,0.728067,...,0.291041,-0.816391,-0.269968,0.294042,0.184222,0.102169,-0.091690,0.376720,0.133610,4
2,Changelog for OkHttp 4.9.2,0.953053,0.351373,0.209267,-0.106836,-0.125029,-0.198518,0.000639,0.419817,0.506266,...,0.396632,-0.125271,-0.063867,-0.245790,-0.077856,-0.362197,0.382095,0.177644,-0.082190,9
3,JDK 17 CI Build (#6864),0.436230,-0.373104,0.590707,-0.170037,0.447702,-0.202234,0.367001,0.848642,0.112943,...,-0.163423,-0.383527,-0.250281,0.552449,0.674672,-0.433067,-0.010147,0.866306,-0.400598,4
4,Update dependencies including kotlin 1.5.31 (#...,0.549877,-0.165462,0.207532,0.250959,-0.002015,0.240586,-0.002104,0.636788,1.031560,...,-0.098021,0.062969,-0.263597,-0.268875,0.287642,-0.568436,0.227022,0.430540,0.042340,4


In [215]:
group_list = sorted(commit_message_vecs_hdf.kmean_label.unique())
for group_id in group_list:
    mask = commit_message_vecs_hdf.kmean_label == group_id
    print('group_id = %d' % group_id)
    min_size = min(sum(mask), 10)
    print( commit_message_vecs_hdf[mask].sample(min_size) )

group_id = 0
                                                message  message_C1  \
2088  Test for OpenJSSE (#5348)\r\n\r\n* Test for Op...    0.442931   
1785  [maven-release-plugin] prepare release parent-1.2    0.529061   
1043  New failing test for HTTP/2 connection interfe...    0.060109   
71                Let's encrypt regression test (#6423)    0.308960   
1992                          Fix tests for Android 5.0    0.728064   
465   Platform cleanup - move trust manager building...    0.655490   
72                     Android Additional Tests (#6420)    0.761778   
351   Fix okHttp for RoboVM (and other Android-based...    0.506234   
515                    Fix link for provided dependency    0.560376   
462   [maven-release-plugin] prepare release parent-...    0.529061   

      message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
2088   -0.238492    0.175596   -0.042845    0.266187   -0.316319    0.169266   
1785   -0.323492   -0.239767    0.435325    0

[10 rows x 32 columns]
group_id = 3
                                                message  message_C1  \
257   Upgrade to Okio 1.17.1.\r\n\r\nUse the new Pip...   -0.100406   
508      Disallow Interceptor responses with no bodies.    0.312648   
753   Clarify comparison test for a change in header...    0.477811   
497   Add support for connect start/end events. (#3434)    0.097837   
784               Upgrade to Okio 1.9 and use its pipe.   -0.001662   
1612                     Fix 1.5 doc and download link.    0.653092   
1577  Update Response bytes() to use Okio.\r\n\r\nDe...    0.085485   
199     Add missing license header to CacheControl.java    0.662659   
1493  Allow optional content-type for RequestBody.cr...    0.419605   
746   Merge branch 'artem-zinnatullin-response-body-...    0.445886   

      message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
257    -0.101755   -0.053063    0.311816    0.073397   -0.147587   -0.000866   
508     0.219563    0.

[10 rows x 32 columns]
group_id = 6
                                                message  message_C1  \
706   Use web socket close codes 1005 and 1006 corre...    0.116170   
1479      Use a better message for impossible requests.    0.253744   
645   Fix RealConnection to guard allocationLimit by...    0.060820   
1597  Teach MockWebServer to throttle the request bo...    0.198834   
1211  Do IPv6 parsing natively.\r\n\r\nAvoid any ris...    0.075778   
1564             Use a ForwardingSource where possible.   -0.110426   
1114  Fold MockWebServerRule into MockWebServer.\r\n...    0.183012   
2014  change the order of two parameters\r\n\r\nThe ...    0.104940   
1778  Hide more flags from SPDY APIs.\r\n\r\nFlags a...   -0.014635   
1174  Clean up SSL state in OkHttpClient (#5949)\r\n...    0.232066   

      message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
706     0.023164    0.356755    0.293295    0.156610   -0.249699    0.039530   
1479    0.164946    0.

[10 rows x 32 columns]
group_id = 9
                                                message  message_C1  \
1603  Update changelog for OkHttp 1.5.2.\r\n\r\n(che...    0.508381   
1401                 Update the changelog for 2.1.0-RC1    0.637247   
469                        More changelog copy editing.    0.605072   
865                 Fix a typo in the interceptor docs.    0.490506   
932   Update okhttp-logging-interceptor README.md\r\...    0.356711   
2173                              Update documentations    0.467417   
1994  Update the changelog for OkHttp 3.12.7 and 3.14.5    0.557760   
2                            Changelog for OkHttp 4.9.2    0.953053   
2150  Add syntax highlighting to code snippets in HT...    0.637396   
768                              Update Javadoc example    0.665028   

      message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
1603    0.057467    0.209439   -0.233440    0.024940   -0.057528    0.099039   
1401    0.019906    0.

In [216]:
okhttp_commits_df['kmean_label'] = commit_message_vecs_hdf['kmean_label']
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring,kmean_label
0,3ef564b0ddcb00655e540e52ad882a5881c1dc54,Jesse Wilson,jesse@swank.ca,2021-11-01 13:29:31,swankjesse,Don't document mockwebserver3.internal (#6895),1ed9863131c87420381d8a6be48b96157a9bb764,2021-10-29 07:39:56,77.0,0,3.0
1,1ed9863131c87420381d8a6be48b96157a9bb764,Jesse Wilson,jesse@swank.ca,2021-10-29 07:39:56,swankjesse,Upgrade to Okio 3.0.0 (#6896),bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738,2021-10-22 16:51:49,158.0,0,4.0
4,a87f4b4585fe6e8e49c7760986fd5d2c591e0573,Jesse Wilson,jwilson@squareup.com,2021-10-01 01:13:35,swankjesse,Changelog for OkHttp 4.9.2,286187102848b30d9580794795746b28462a79d0,2021-09-24 06:13:58,162.0,0,4.0
5,286187102848b30d9580794795746b28462a79d0,Yuri Schimke,yuri@schimke.ee,2021-09-24 06:13:58,yschimke,JDK 17 CI Build (#6864),980dfbbe9d0e276b0a103df373f8969e21667e50,2021-09-20 19:37:03,82.0,0,4.0
6,980dfbbe9d0e276b0a103df373f8969e21667e50,Goooler,wangzongler@gmail.com,2021-09-20 19:37:03,Goooler,Update dependencies including kotlin 1.5.31 (#...,6f328ecab46e070345bf6f4de86b98a24e50e3d8,2021-09-16 06:33:06,109.0,0,3.0


In [217]:
okhttp_commits_df[okhttp_commits_df['contains_refactoring'] == 1]

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring,kmean_label
150,ff959f8fd550b8f55a47038ea4235b1b2ed2eaa8,Yuri Schimke,yuri@schimke.ee,2020-11-15 06:42:22,yschimke,Android Additional Tests (#6420),4adcb33d1b859e589e929629232e2e85092948d2,2020-11-14 17:38:22,13.0,1,7.0
191,59ae0c138d6468aac5d92781b19cddc26e04e862,Jesse Wilson,jesse@swank.ca,2020-11-01 05:46:35,swankjesse,Convert more tests to JUnit 5 (#6366),963c76864344d0d55ba7421f474c130a958a9649,2020-10-31 22:51:34,6.0,1,8.0
192,963c76864344d0d55ba7421f474c130a958a9649,Jesse Wilson,jesse@swank.ca,2020-10-31 22:51:34,swankjesse,Migrate all OkHttpClientTestRule uses to JUnit...,559b0f8d4b771e30ea27cd5712f55ae092f85aca,2020-10-31 13:39:47,9.0,1,6.0
1186,f8065acac28a0b86f3bdbd854ac87af0789416d0,Jesse Wilson,jesse@swank.ca,2020-10-30 06:59:02,swankjesse,Execute CallTest with JUnit 5 (#6349)\n\n* Exe...,f1bc9d1d3339d0696e232a5d8a2d7830630a8242,2020-10-24 04:03:17,146.0,1,2.0


In [218]:
okhttp_commits_df.groupby('kmean_label').agg({'time_taken': ['mean', 'min', 'max']})

time_taken            
                  mean  min    max
kmean_label                       
0.0          39.051546  6.0  164.0
1.0          44.396341  6.0  161.0
2.0          43.065000  6.0  166.0
3.0          42.712000  6.0  160.0
4.0          55.074074  7.0  162.0
5.0          39.459016  6.0  145.0
6.0          37.859438  6.0  167.0
7.0          39.307692  6.0  140.0
8.0          38.340000  6.0  166.0
9.0          44.244444  7.0  147.0